keras과제에서 구현한 model_5 를 


pytorch로 구현

(Adam사용, keras callback함수로 사용했던것을 제외)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls


drive/  sample_data/


In [18]:
cd /content/drive/My Drive/tobigs_7HW

/content/drive/My Drive/tobigs_7HW


In [19]:
ls

'7_HW(keras_CNN).ipynb'                         submission_2D_47_re.csv
'7_HW(keras).ipynb'                             submission_2D_60.csv
'7_HW(pytorch).ipynb'                           submission_2D.csv
 checkpoint-epoch-150-batch-1024-trial-001.h5   submission_2D_model3.csv
'kaggle (1).json'                               submission_2D_model4.csv
'kaggle (2).json'                               submission_2D_model5.csv
'kaggle (3).json'                               submission_2D_model6.csv
'kaggle (4).json'                               submission_2D_model7.csv
'kaggle (5).json'                               submission.csv
 kaggle.json                                    submission_l9_adam.csv
 model_5.png                                    test_df.csv
 RAdam.py                                       test_df.csv.zip
 sample_submission.csv                          train_df.csv
 submission_2D_47.csv                           train_df.csv.zip


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import itertools
from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

%matplotlib inline

In [0]:
sample_submission = pd.read_csv("sample_submission.csv")

In [69]:
test = pd.read_csv("test_df.csv")
train = pd.read_csv("train_df.csv")

print(f"Train data shape {train.shape}")
print(f"Test data shape {test.shape}")


Train data shape (42000, 785)
Test data shape (18000, 785)


In [0]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
test = test.iloc[:, 1:].values / 255

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch
class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        x2d = X.reshape(28, 28)
        #x2d_aug = aug(x2d)
        x2d = X.reshape(784)
        return torch.tensor(X).float(), torch.tensor(y)

class TestDataset(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X = self.X[idx]
        return torch.tensor(X).float()

In [0]:
traindataset = TrainDataset(X, y)
testdataset = TestDataset(test)

In [0]:
#batch iterator
trainloader = DataLoader(traindataset, batch_size=1024, shuffle=True, num_workers=2)
testloader = DataLoader(testdataset, batch_size=1024, shuffle=False, num_workers=2)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

def swish(x):
  return x * F.sigmoid(x)


class Net(nn.Module):
    def __init__(self,dropout = 0.2):
        super(Net, self).__init__()
        self.dropout = dropout #drop out 쓸꼬양
        

        self.bn1 = nn.BatchNorm1d(num_features=128)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.bn3 = nn.BatchNorm1d(num_features=128)

        self.d1_1 = nn.Dropout(p=self.dropout)
        self.d1_2 = nn.Dropout(p=self.dropout)
        self.d1_3 = nn.Dropout(p=self.dropout)

        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)


        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = swish(x)
        x = self.d1_1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = swish(x)
        x = self.d1_2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = swish(x)
        x = self.d1_3(x)

        x = self.fc4(x)
        x = swish(x)

        x = self.fc5(x)


        return F.log_softmax(x, dim=-1)

net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [76]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)   

cuda


In [0]:
net = net.to(device)

In [79]:
num_epochs = 50
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
    running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,    42] loss: 4.702
[2,    42] loss: 4.598
[3,    42] loss: 4.496
[4,    42] loss: 4.393
[5,    42] loss: 4.278
[6,    42] loss: 4.167
[7,    42] loss: 4.055
[8,    42] loss: 3.948
[9,    42] loss: 3.834
[10,    42] loss: 3.733
[11,    42] loss: 3.623
[12,    42] loss: 3.520
[13,    42] loss: 3.422
[14,    42] loss: 3.325
[15,    42] loss: 3.222
[16,    42] loss: 3.135
[17,    42] loss: 3.041
[18,    42] loss: 2.956
[19,    42] loss: 2.871
[20,    42] loss: 2.781
[21,    42] loss: 2.704
[22,    42] loss: 2.632
[23,    42] loss: 2.550
[24,    42] loss: 2.473
[25,    42] loss: 2.410
[26,    42] loss: 2.347
[27,    42] loss: 2.284
[28,    42] loss: 2.207
[29,    42] loss: 2.144
[30,    42] loss: 2.069
[31,    42] loss: 2.026
[32,    42] loss: 1.959
[33,    42] loss: 1.908
[34,    42] loss: 1.840
[35,    42] loss: 1.793
[36,    42] loss: 1.744
[37,    42] loss: 1.692
[38,    42] loss: 1.651
[39,    42] loss: 1.615
[40,    42] loss: 1.551
[41,    42] loss: 1.509
[42,    42] loss: 1.471
[

In [82]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())
print(preds[:10])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[array(8), array(0), array(5), array(3), array(8), array(1), array(9), array(6), array(6), array(0)]
